In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [3]:
appid = 271590
URL = f"https://store.steampowered.com/app/{appid}"

res = requests.get(URL)
soup = BeautifulSoup(res.content, 'html.parser')


In [8]:
detail = soup.select_one("#genresAndManufacturer").text

In [17]:
detail_temp = detail.split("\n")

detail_list = []
for dt in detail_temp:
    detail_list.append(dt.split(":"))

for i in range(len(detail_list)):
    if detail_list[i][0] == "Title":
        title = detail_list[i][1]
    elif detail_list[i][1:] == "Genre":
        genre = detail_list[i][1]
    elif detail_list[i][0] == "Developer":
        developer = detail_list[i+1][0]
    elif detail_list[i][0] == "Publisher":
        publisher = detail_list[i+1][0]
    elif detail_list[i][0] == "Franchise":
        franchise = detail_list[i+1][0]
    elif detail_list[i][0] == "Release Date":
        release_date = detail_list[i][1]

try: 
    publisher
except:
    publisher = "0"

try: 
    franchise
except:
    franchise = "0"

try:
    recent_reviews = soup.select_one("#userReviews > div:nth-child(1) > div.summary.column > span").text
except:
    recent_reviews = "0"
    
try:    
    recent_reviews_temp = soup.select_one('#userReviews > div:nth-child(1) > div.summary.column > span.nonresponsive_hidden.responsive_reviewdesc')
    # recent_reviews 평가 비율(최근 30일)
except:
    recent_reviews_ratio = "0"
    recent_reviews_voted_users = "0"
try:
    recent_reviews_ratio = soup.select_one("#userReviews > div:nth-child(1) > div.summary.column > span.nonresponsive_hidden.responsive_reviewdesc").text.split()[1].replace("%","")
    if recent_reviews_ratio == "Need":
        recent_reviews_ratio = "0"
        
except:
    recent_reviews_ratio = "0"
    # recent_reviews 점수를 준 유저 수(최근 30일)
try:
    recent_reviews_voted_users = soup.select_one("#userReviews > div:nth-child(1) > div.summary.column > span.nonresponsive_hidden.responsive_reviewdesc").text.split()[4].replace(",","")
    if recent_reviews_voted_users == "reviews":
        recent_reviews_voted_users = "0"
except:
    recent_reviews_voted_users = "0"
    
# 전체기간 유저 평가 신작 게임은 없는 경우가 있다.
try:
    all_reviews = soup.select_one("#userReviews > div:nth-child(2) > div.summary.column > span.game_review_summary.positive").text
except:
    all_reviews = "0"
    
try:
    all_reviews_temp = soup.select_one('#userReviews > div:nth-child(2) > div.summary.column > span.nonresponsive_hidden.responsive_reviewdesc')
except:
    all_reviews_ratio = "0"
    all_reviews_voted_users = "0"
try:
    # 전체기간 유저 평가 비율
    all_reviews_ratio = soup.select_one("#userReviews > div:nth-child(2) > div.summary.column > span.nonresponsive_hidden.responsive_reviewdesc").text.split()[1].replace("%", "")
except:
    all_reviews_ratio = "0"
    # 전체기간 유저 점수를 준 유저 수
try:
    all_reviews_voted_users = soup.select_one("#userReviews > div:nth-child(2) > div.summary.column > span.nonresponsive_hidden.responsive_reviewdesc").text.split()[4].replace(",","")
except:
    all_reviews_voted_users = "0"

#image link
try:
    image_link = soup.select_one("img.game_header_image_full")["src"]
except:
    image_link = "0"

# grade
try:
    grade = soup.select_one("div.game_rating_icon > img")["src"].split("/")[-1].split(".")[0]
except:
    grade = "0"

# info
try:
    info_group = soup.select("div.label")
    info_list = [i.text for i in info_group]
except:
    info_list = ["0"]

# tag
try:
    tag_group = soup.select("a.app_tag")
    tag_list = [i.text for i in info_group]
except:
    tag_list = ["0"]

# about_this_game
try:
    about_this_game = soup.select_one("#game_area_description").text.replace("\n\n"," ").replace("\n", " ")[16:].strip()
except:
    about_this_game = "0"

In [104]:
language_group = soup.select("#languageTable > table > tr")
parsed_language = language_group[1:]
parsed_language

[<tr class="" style="">
 <td class="ellipsis" style="width: 94px; text-align: left">
 				English			</td>
 <td class="checkcol">
 <span>✔</span> </td>
 <td class="checkcol">
 <span>✔</span> </td>
 <td class="checkcol">
 <span>✔</span> </td>
 </tr>,
 <tr class="" style="">
 <td class="ellipsis" style="width: 94px; text-align: left">
 				French			</td>
 <td class="checkcol">
 <span>✔</span> </td>
 <td class="checkcol">
 </td>
 <td class="checkcol">
 <span>✔</span> </td>
 </tr>,
 <tr class="" style="">
 <td class="ellipsis" style="width: 94px; text-align: left">
 				Italian			</td>
 <td class="checkcol">
 <span>✔</span> </td>
 <td class="checkcol">
 </td>
 <td class="checkcol">
 <span>✔</span> </td>
 </tr>,
 <tr class="" style="">
 <td class="ellipsis" style="width: 94px; text-align: left">
 				German			</td>
 <td class="checkcol">
 <span>✔</span> </td>
 <td class="checkcol">
 </td>
 <td class="checkcol">
 <span>✔</span> </td>
 </tr>,
 <tr class="" style="">
 <td class="ellipsis" style

In [105]:
language_list = []
for i in parsed_language:
    language_list.append(i.select_one("td.ellipsis").text.strip())

In [147]:
parsed_language[0]

<tr class="" style="">
<td class="ellipsis" style="width: 94px; text-align: left">
				English			</td>
<td class="checkcol">
<span>✔</span> </td>
<td class="checkcol">
<span>✔</span> </td>
<td class="checkcol">
<span>✔</span> </td>
</tr>

In [149]:
interface_list = []
for i in range(len(language_list)):
    if parsed_language[i].select_one("td.checkcol > span").text == "✔":
        interface_list.append(language_list[i])

print(interface_list)


['English', 'French', 'Italian', 'German', 'Spanish - Spain', 'Korean', 'Polish', 'Portuguese - Brazil', 'Russian', 'Traditional Chinese', 'Japanese', 'Simplified Chinese', 'Spanish - Latin America']


In [151]:
fullaudio_list = []
for i in range(len(language_list)):
    if parsed_language[i].select_one("#languageTable > table > tbody > tr:nth-child() > td:nth-child(3)").text == "✔":
        fullaudio_list.append(language_list[i])

print(fullaudio_list)




SelectorSyntaxError: Invalid syntax for pseudo class ':nth-child'
  line 1:
#languageTable > table > tbody > tr:nth-child() > td:nth-child(3)
                                   ^

In [144]:
parsed_language[0].select_one("td.checkcol > span").text

'✔'

In [109]:
interface_list = []
for i in range(len(language_list)):
    try:
        if parsed_language[i].select_one("td.checkcol > span").text == '✔':
            # interface_list에 해당 언어 c추가
            interface_list.append(language_list[i])
    except:
        continue

fullaudio_list = []
for i in range(len(language_list)):
    try:
        if parsed_language[i].select_one("#languageTable > table > tbody > tr > td:nth-child(3)").text == '✔':
            fullaudio_list.append(language_list[i])
    except:
        continue
    
# 아무것도 안들어잇으면 0 넣어주기
if len(fullaudio_list) == 0:
    fullaudio_list.append("0")

subtitles_list = []
for i in range(len(language_list)):
    try:
        if parsed_language[i].select_one("#languageTable > table > tbody > tr > td:nth-child(4)").text == '✔':
            subtitles_list.append(language_list[i])
    except:
        continue

# # 아무것도 안들어잇으면 0 넣어주기
# if len(subtitles_list) == 0:
#     subtitles_list = "0"
subtitles_list

[]